In [117]:
#Part of code taken from https://learnopencv.com/weighted-boxes-fusion/

import torch 
import cv2 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from PIL import Image
import os
from ensemble_boxes import *
from dotenv import load_dotenv

In [118]:
model_1=torch.hub.load("ultralytics/yolov5",'custom',path='best_yolov5_weights.pt',device=torch.device('mps'))
model_2=torch.hub.load("ultralytics/yolov5",'custom',path='last.pt',device=torch.device('mps'))

Using cache found in /Users/smudge/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-28 Python-3.11.4 torch-2.1.0 MPS

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Using cache found in /Users/smudge/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-28 Python-3.11.4 torch-2.1.0 MPS

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [119]:
#set the env variables
load_dotenv('./.env')
PATH_TO_TEST=os.getenv("PATH_TO_TEST")
PATH_TO_INPAINT=os.getenv("PATH_TO_INPAINT")
PATH_TO_AUTO=os.getenv("PATH_TO_AUTO")


In [120]:
#Get the image ids
test_ids=os.listdir(PATH_TO_TEST)
inpaint_ids=os.listdir(PATH_TO_INPAINT)
auto_ids=os.listdir(PATH_TO_AUTO)

In [121]:
#get the predictions

def get_predictions(img_names,path,aug,model):

    model.conf=0.25

    pred_scores=dict()
    pred_boxes=dict()
    pred_classes=dict()

    for img in img_names:
        if '_' not in img:
            img_path=path+img

            prediction=model(img_path,size=1024,augment=aug)

            pred_scores[img]=prediction.xyxy[0][:,4].to(device=torch.device('cpu')).detach().numpy()
            pred_boxes[img]=prediction.xyxy[0][:,0:4].to(device=torch.device('cpu')).detach().numpy()
            pred_classes[img]=prediction.xyxy[0][:,5].to(device=torch.device('cpu')).detach().numpy()
    
    return pred_scores,pred_boxes,pred_classes

In [122]:
#get predictions for different test sets

pred_confs=[]
pred_boxes=[]
pred_classes=[]

test_paths=[PATH_TO_TEST,PATH_TO_INPAINT,PATH_TO_AUTO]
test_imgids=[test_ids,inpaint_ids,auto_ids]
models=[model_1,model_2]


for i in range(2):
    considered_model=models[i]
    for j in range(3):
        confs_scores,box_preds,cls_pred=get_predictions(test_imgids[j],test_paths[j],aug=False,model=considered_model)

        pred_confs.append(confs_scores)
        pred_boxes.append(box_preds)
        pred_classes.append(cls_pred)
    

In [123]:
#function for weighted box fusion
def perform_wbf(pred_confs_models,pred_boxes_models,pred_classes_models):
    wbf_boxes_dict=dict()
    wbf_scores_dict=dict()

    for image_id in test_ids:
        if '_' not in image_id:
            res_array=np.array([1024,1024,1024,1024]) #1024X1024 images

            all_model_boxes=[]
            all_model_scores=[]
            all_model_classes=[]

            for boxes,scores,classes in zip(pred_boxes_models,pred_confs_models,pred_classes_models):
                pred_boxes_norm=(boxes[image_id]/res_array).clip(min=0.,max=1.)
                scores_model=scores[image_id]
                classes_model=classes[image_id]

                all_model_boxes.append(pred_boxes_norm)
                all_model_scores.append(scores_model)
                all_model_classes.append(classes_model)
            
            boxes,scores,labels=weighted_boxes_fusion(all_model_boxes,all_model_scores,all_model_classes,weights=None,iou_thr=0.5,skip_box_thr=0.25)

            final_score_ids=np.where(scores>0.26)[0]
            final_boxes=boxes[final_score_ids]
            final_scores=scores[final_score_ids]

            final_boxes=(final_boxes*res_array).clip(min=[0,0,0,0],max=[1023,1023,1023,1023])

            final_boxes=final_boxes.astype("int")

            final_boxes[:,2:]=final_boxes[:,2:]-final_boxes[:,:2]
            wbf_boxes_dict[image_id]=final_boxes.tolist()
            wbf_scores_dict[image_id]=np.expand_dims(np.round(final_scores,5),axis=-1).tolist()
    
    return wbf_boxes_dict,wbf_scores_dict


        

In [124]:
boxes_dict_wbf,scores_dict_wbf=perform_wbf(pred_confs,pred_boxes,pred_classes)

In [125]:
#save outputs to csv file

boxes=list()
ids=list()
scores=list()
for i in boxes_dict_wbf.keys():
    boxes.append(boxes_dict_wbf[i])
    ids.append(i)
    scores.append(scores_dict_wbf[i])

df=pd.DataFrame({
    'image_id':ids,
    'boxes':boxes,
    'scores':scores
})

df.to_csv("./Results/ensemble_test_auto_inpaint_conf_0.25.csv")